In [2]:
import plotly.graph_objects as go
#!pip install plotly chart_studio --upgrade
import chart_studio.plotly as py
#라이브러리 설치방법은 다음과 같다.
# pip install cufflinks
# pip install chart_studio
import cufflinks as cf
cf.go_offline(connected=True)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from textwrap import wrap


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, OPTICS
from scipy.cluster.hierarchy import dendrogram

import re
import string
import time
from PIL import Image
from collections import Counter

import plotly.express as px

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words as nltk_words
from nltk.corpus import stopwords
from matplotlib import font_manager, rc


nltk.download('stopwords', download_dir='./resources')
nltk.download('punkt', download_dir='./resources')
nltk.download('wordnet', download_dir='./resources')
nltk.download('omw-1.4', download_dir='./resources')
nltk.download('words', download_dir='./resources')
nltk.data.path.append('./resources')

[nltk_data] Downloading package stopwords to ./resources...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to ./resources...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to ./resources...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to ./resources...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to ./resources...
[nltk_data]   Package words is already up-to-date!


사용 데이터 <br>
https://www.kaggle.com/datasets/gpreda/bbc-news [BBC News]

In [3]:
data = pd.read_csv('data/BBC_뉴스.csv')
data

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...
...,...,...,...,...,...
10825,World Cup 2022: Switzerland 1-0 Cameroon - Bre...,"Thu, 24 Nov 2022 14:46:06 GMT",https://www.bbc.co.uk/sport/football/63657470,https://www.bbc.co.uk/sport/football/63657470?...,Switzerland's Cameroon-born Breel Embolo score...
10826,World Cup 2022: 'Furious' Wales take LGBTQ+ ri...,"Thu, 24 Nov 2022 05:49:11 GMT",https://www.bbc.co.uk/sport/football/63737709,https://www.bbc.co.uk/sport/football/63737709?...,"Wales are ""furious"" players were threatened wi..."
10827,China asks: Is World Cup on another planet?,"Thu, 24 Nov 2022 10:24:42 GMT",https://www.bbc.co.uk/news/world-asia-china-63...,https://www.bbc.co.uk/news/world-asia-china-63...,Scenes of maskless crowds in Qatar anger viewe...
10828,World Cup 2022: What has and hasn’t been banne...,"Wed, 23 Nov 2022 23:38:52 GMT",https://www.bbc.co.uk/news/world-63735559,https://www.bbc.co.uk/news/world-63735559?at_m...,Are supporters in Qatar managing to keep up wi...


In [4]:
sns.set_style('darkgrid')

In [5]:
STOPWORDS = stopwords.words('english')
pd.Series(STOPWORDS)

0             i
1            me
2            my
3        myself
4            we
         ...   
174     weren't
175         won
176       won't
177      wouldn
178    wouldn't
Length: 179, dtype: object

In [6]:
# BBC 언급 단어 Top15 추출
top_N = 15
a = data['title'].str.lower().str.replace(r'[^\w\s]', '', regex=True).str.cat(sep=' ')
words = list(filter(lambda w: w not in STOPWORDS, nltk.tokenize.word_tokenize(a)))
word_dist = nltk.FreqDist(words)
freq_title = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
freq_title

,Word,Frequency
0,ukraine,991
1,war,673
2,2022,520
3,world,499
4,england,490
5,says,453
6,uk,414
7,new,378
8,cup,367
9,us,319


In [7]:
freq_title.set_index('Word', inplace=True)

In [11]:
freq_title[::-1].iplot(kind='bar', orientation="h",title='BBC 최다 언급 단어 순위')